In [1]:
# Web Scraper

# Imports
import os  #manipular o sistema operacional
import re  #expressão regular que usa-se pra buscar padrões nos dados
import csv #pra gravar em csv o resultado
import pickle #para criar um cache de acesso a pagina web
import requests # para fazer requisição a pagina
from bs4 import BeautifulSoup # pra fazer o parser do código html
from urllib.request import urlopen
from pprint import pprint
import urllib3
import pandas as pd


In [2]:
url = 'https://lista.mercadolivre.com.br/celular#D[A:celular]'

results = requests.get(url)

texto_web = results.text

In [14]:
soup = BeautifulSoup(texto_web,'html.parser')

In [28]:
def get_link_pages(soup):
    try:
        links = soup.findAll('a', class_='andes-pagination__link ui-search-link')
        urls = [item.get('href') for item in links]
    except:
        linK = 'não foi possível'
    return urls
get_link_pages(soup)    

['https://celulares.mercadolivre.com.br/',
 'https://celulares.mercadolivre.com.br/_Desde_51',
 'https://celulares.mercadolivre.com.br/_Desde_101',
 'https://celulares.mercadolivre.com.br/_Desde_151',
 'https://celulares.mercadolivre.com.br/_Desde_201',
 'https://celulares.mercadolivre.com.br/_Desde_251',
 'https://celulares.mercadolivre.com.br/_Desde_301',
 'https://celulares.mercadolivre.com.br/_Desde_351',
 'https://celulares.mercadolivre.com.br/_Desde_401',
 'https://celulares.mercadolivre.com.br/_Desde_451',
 'https://celulares.mercadolivre.com.br/_Desde_51']

In [439]:
li = 'Smartphone Xiaomi Poco M3 Dual SIM 64 GB  cool blue 4 GB RAM'
info_nome_marca=li.strip().split()[1:4]
info_nome_marca

['Xiaomi', 'Poco', 'M3']

In [422]:
li = 'Celular Celular Xiaomi Poco M3 Dual SIM 64 GB  cool blue 4 GB RAM'
if ((li.strip().split()[0]== 'Celular') or (li.strip().split()[0]== 'Smartphone') ):
    info_nome_marca=li.strip().split()[1]
    if ((info_nome_marca=='Smartphone') or (info_nome_marca=='Celular')):
        info_nome_marca=li.strip().split()[2]
    else:
        info_nome_marca=info_nome_marca
# else:
#     info_nome_marca=li.strip().split()[0]
info_nome_marca

'Xiaomi'

In [452]:
url = 'https://celulares.mercadolivre.com.br/iphone/#origin=brands_carousel&position=0&id=9344&applied_filter_id%3DBRAND%26applied_filter_name%3DMarca%26applied_filter_order%3D-1%26applied_value_id%3D9344%26applied_value_name%3DApple%26applied_value_order%3D0%26applied_value_results%3D32380'

bloco_mercado_livre_Apple = []

results = requests.get(url)

texto_web = results.text

soup = BeautifulSoup(texto_web,'html.parser')

urls = get_link_pages(soup) 


for link in urls:    
    
    page = link

    results = requests.get(page)

    texto_web = results.text

    soup = BeautifulSoup(texto_web,'html.parser')

    
    for info in soup.findAll('div', {'class':'ui-search-result__content-wrapper'}):
        bloco = []

       
        # Nome e específicações
        info_nome = info.find('a').attrs['title']
        # Preço
        info_cent = info.find('span', {'class':'price-tag-fraction'})
       
    
    
        # Ram do aparelho
        info_ram = re.findall(r'\d+?[\s+\d+]?[gGmM][bB][\s+\w][rR][aA][mM]', info_nome, re.M|re.I)


        info_sem_ram = re.sub(r'\d+?[\s+\d+][gGmM][bB][\s+\w][rR][aA][mM]', "", info_nome, re.M|re.I)
        info_sem_marca = re.sub(r'^\w+', "" ,info_sem_ram, re.M|re.I)
        info_dual_sim = re.findall(r'Dual\sSIM',info_sem_marca, re.M|re.I)
        if info_dual_sim:
            info_dual_sim = ['2']
        else:
            info_dual_sim = ['1']

        if ((info_nome.strip().split()[0]== 'Celular') or (info_nome.strip().split()[0]== 'Smartphone') ):
            info_nome_marca=info_nome.strip().split()[1]
            if ((info_nome_marca=='Smartphone') or (info_nome_marca=='Celular')):
                info_nome_marca=info_nome.strip().split()[2]
            else:
                info_nome_marca=info_nome_marca

        
        info_cor = info_nome.strip().split()[-4]
    
          
        
        info_sem_dual = re.sub(r'Dual\sSIM', "" ,info_sem_marca, re.M|re.I)
        info_hd = re.findall(r'\d+[\w\s][gGmM][bB]',info_sem_dual, re.M|re.I )
        info_sem_hd = re.sub(r'\d+[\w\s][gGmB][bB]', "", info_sem_dual, re.M|re.I )
       
        info_sem_cor = re.sub(r'\w+[-]?\w+[\s]?$', "",info_sem_hd, re.M|re.I )

        # Nome e específicações
    #     bloco.append(str(info_nome))
    #     print('Nome e específicações:', info_nome)

#         bloco.append(info_nome)
#         print('info:', info_nome)


#         print('info_sem_hd:', info_sem_hd)
#         # Informações sem ram, marca, dual e hd
         
#         "Variable as string = %s" %(variable)
        bloco.append("Iphone %s" %(info_nome))
        print("Nome do Celular:Iphone %s" %(info_nome))
        
        print(info_nome)

        # Marca do celular
        bloco.append('Apple')
        print('Marca do celular:', 'Apple')


        #Cor do Celular
        bloco.append(info_cor.title())
        print('Cor:', info_cor.title())


        # Memória ram
        bloco.append(info_ram)
        print('Memória ram:', info_ram)

        # Espaço de hd
        bloco.append(info_hd)
        print('Espaço em hd:', str(info_hd).title().strip())


        # 1/2 ship(s)
        bloco.append(info_dual_sim)
        print('1 ou 2 ship(s):', info_dual_sim)


        # Preço do celualr
        bloco.append(info_cent.text)
        print('Preço:', info_cent.text)
        print('=======================================')

        # lista mestre
        bloco_mercado_livre_Apple.append(bloco)

       
    
pprint(bloco_mercado_livre_Apple)     


Nome do Celular:Iphone iPhone 11 128 GB Preto
iPhone 11 128 GB Preto
Marca do celular: Apple
Cor: 11
Memória ram: []
Espaço em hd: ['128 Gb']
1 ou 2 ship(s): ['1']
Preço: 5.506
Nome do Celular:Iphone iPhone XR 64 GB  preto
iPhone XR 64 GB  preto
Marca do celular: Apple
Cor: Xr
Memória ram: []
Espaço em hd: ['64 Gb']
1 ou 2 ship(s): ['1']
Preço: 3.880
Nome do Celular:Iphone iPhone 6s 64 GB  cinza-espacial
iPhone 6s 64 GB  cinza-espacial
Marca do celular: Apple
Cor: 6S
Memória ram: []
Espaço em hd: ['64 Gb']
1 ou 2 ship(s): ['1']
Preço: 1.750
Nome do Celular:Iphone iPhone 11 64 GB (Product)Red
iPhone 11 64 GB (Product)Red
Marca do celular: Apple
Cor: 11
Memória ram: []
Espaço em hd: ['64 Gb']
1 ou 2 ship(s): ['1']
Preço: 4.749
Nome do Celular:Iphone iPhone 12 Pro Max 128 GB  ouro
iPhone 12 Pro Max 128 GB  ouro
Marca do celular: Apple
Cor: Max
Memória ram: []
Espaço em hd: ['128 Gb']
1 ou 2 ship(s): ['1']
Preço: 9.190
Nome do Celular:Iphone iPhone 7 Plus 128 GB ouro
iPhone 7 Plus 128 GB o

Nome do Celular:Iphone iPhone XR 64 Gb Azul - Usado + Garantia + Brinde
iPhone XR 64 Gb Azul - Usado + Garantia + Brinde
Marca do celular: Apple
Cor: +
Memória ram: []
Espaço em hd: ['64 Gb']
1 ou 2 ship(s): ['1']
Preço: 2.999
Nome do Celular:Iphone Apple iPhone 8 Usado Vitrine 64gb Dourado Com Garantia 
Apple iPhone 8 Usado Vitrine 64gb Dourado Com Garantia 
Marca do celular: Apple
Cor: 64Gb
Memória ram: []
Espaço em hd: ['64Gb']
1 ou 2 ship(s): ['1']
Preço: 2.299
Nome do Celular:Iphone Apple iPhone 6s Vitrine 32gb Original Barato 12x Sem Juros
Apple iPhone 6s Vitrine 32gb Original Barato 12x Sem Juros
Marca do celular: Apple
Cor: Barato
Memória ram: []
Espaço em hd: ['32Gb']
1 ou 2 ship(s): ['1']
Preço: 1.365
Nome do Celular:Iphone iPhone 7 Plus 32 Gb Tela 5,5 Apple 4k Promocão - Vitrine
iPhone 7 Plus 32 Gb Tela 5,5 Apple 4k Promocão - Vitrine
Marca do celular: Apple
Cor: 4K
Memória ram: []
Espaço em hd: ['32 Gb']
1 ou 2 ship(s): ['1']
Preço: 2.147
Nome do Celular:Iphone Apple iPhone

IndexError: list index out of range

In [427]:

url = 'https://lista.mercadolivre.com.br/celular#D[A:celular]'

bloco_mercado_livre = []

results = requests.get(url)

texto_web = results.text

soup = BeautifulSoup(texto_web,'html.parser')

urls = get_link_pages(soup) 


for link in urls:    
    
    page = link

    results = requests.get(page)

    texto_web = results.text

    soup = BeautifulSoup(texto_web,'html.parser')

    
    for info in soup.findAll('div', {'class':'ui-search-result__content-wrapper'}):
        bloco = []

       
        # Nome e específicações
        info_nome = info.find('a').attrs['title']
        # Preço
        info_cent = info.find('span', {'class':'price-tag-fraction'})
       
    
    
        # Ram do aparelho
        info_ram = re.findall(r'\d+?[\s+\d+]?[gGmM][bB][\s+\w][rR][aA][mM]', info_nome, re.M|re.I)


        info_sem_ram = re.sub(r'\d+?[\s+\d+][gGmM][bB][\s+\w][rR][aA][mM]', "", info_nome, re.M|re.I)
        info_sem_marca = re.sub(r'^\w+', "" ,info_sem_ram, re.M|re.I)
        info_dual_sim = re.findall(r'Dual\sSIM',info_sem_marca, re.M|re.I)
        if info_dual_sim:
            info_dual_sim = ['2']
        else:
            info_dual_sim = ['1']

        if ((info_nome.strip().split()[0]== 'Celular') or (info_nome.strip().split()[0]== 'Smartphone') ):
            info_nome_marca=info_nome.strip().split()[1]
            if ((info_nome_marca=='Smartphone') or (info_nome_marca=='Celular')):
                info_nome_marca=info_nome.strip().split()[2]
            else:
                info_nome_marca=info_nome_marca

        
        info_cor = info_nome.strip().split()[-4]
    
          
        
        info_sem_dual = re.sub(r'Dual\sSIM', "" ,info_sem_marca, re.M|re.I)
        info_hd = re.findall(r'\d+[\w\s][gGmM][bB]',info_sem_dual, re.M|re.I )
        info_sem_hd = re.sub(r'\d+[\w\s][gGmB][bB]', "", info_sem_dual, re.M|re.I )
       
        info_sem_cor = re.sub(r'\w+[-]?\w+[\s]?$', "",info_sem_hd, re.M|re.I )

        # Nome e específicações
    #     bloco.append(str(info_nome))
    #     print('Nome e específicações:', info_nome)

#         bloco.append(info_nome)
#         print('info:', info_nome)


#         print('info_sem_hd:', info_sem_hd)
#         # Informações sem ram, marca, dual e hd

        bloco.append(str(info_sem_cor).title().strip())
        print('Nome do Celular:', str(info_sem_cor).title().strip())
        
        print(info_nome)

        # Marca do celular
        bloco.append(info_nome_marca.title())
        print('Marca do celular:', info_nome_marca.title())


        #Cor do Celular
        bloco.append(info_cor.title())
        print('Cor:', info_cor.title())


        # Memória ram
        bloco.append(info_ram)
        print('Memória ram:', info_ram)

        # Espaço de hd
        bloco.append(info_hd)
        print('Espaço em hd:', str(info_hd).title().strip())


        # 1/2 ship(s)
        bloco.append(info_dual_sim)
        print('1 ou 2 ship(s):', info_dual_sim)


        # Preço do celualr
        bloco.append(info_cent.text)
        print('Preço:', info_cent.text)
        print('=======================================')

        # lista mestre
        bloco_mercado_livre.append(bloco)

       
    
pprint(bloco_mercado_livre)     


Nome do Celular: Nokia C2 Nk011 Tl 5,7   (16+16 Cartão)1Gb
Smartphone Nokia C2 Nk011 Tl 5,7  32gb (16+16 Cartão)1gb Ram
Marca do celular: Nokia
Cor: 32Gb
Memória ram: ['1gb Ram']
Espaço em hd: ['32Gb']
1 ou 2 ship(s): ['1']
Preço: 639
Nome do Celular: Galaxy A10S
Samsung Galaxy A10s Dual SIM 32 GB  preto 2 GB RAM
Marca do celular: Nokia
Cor: Preto
Memória ram: ['2 GB RAM']
Espaço em hd: ['32 Gb']
1 ou 2 ship(s): ['2']
Preço: 839
Nome do Celular: E6 Play
Moto E6 Play Dual SIM 32 GB  cinza-metálico 2 GB RAM
Marca do celular: Nokia
Cor: Cinza-Metálico
Memória ram: ['2 GB RAM']
Espaço em hd: ['32 Gb']
1 ou 2 ship(s): ['2']
Preço: 810
Nome do Celular: Redmi Note 9S
Xiaomi Redmi Note 9S Dual SIM 128 GB azul 4 GB RAM
Marca do celular: Nokia
Cor: Azul
Memória ram: ['4 GB RAM']
Espaço em hd: ['128 Gb']
1 ou 2 ship(s): ['2']
Preço: 1.735
Nome do Celular: Redmi 9A
Xiaomi Redmi 9A Dual SIM 32 GB  cinza 2 GB RAM
Marca do celular: Nokia
Cor: Cinza
Memória ram: ['2 GB RAM']
Espaço em hd: ['32 Gb']
1 

Nome do Celular: E7 Plus    Bronze
Moto E7 Plus Dual SIM 64 GB  bronze âmbar 4 GB RAM
Marca do celular: Samsung
Cor: Âmbar
Memória ram: ['4 GB RAM']
Espaço em hd: ['64 Gb']
1 ou 2 ship(s): ['2']
Preço: 1.078
Nome do Celular: Redmi 9 (Global)   Carbon
Xiaomi Redmi 9 (Global) Dual SIM 32 GB carbon grey 3 GB RAM
Marca do celular: Samsung
Cor: Grey
Memória ram: ['3 GB RAM']
Espaço em hd: ['32 Gb']
1 ou 2 ship(s): ['2']
Preço: 988
Nome do Celular: Flip Up  32 Mb
Multilaser Flip Up Dual SIM 32 MB  preto 32 MB RAM
Marca do celular: Samsung
Cor: Preto
Memória ram: ['32 MB RAM']
Espaço em hd: ['32 Mb']
1 ou 2 ship(s): ['2']
Preço: 155
Nome do Celular: W201  32 Mb  Preto/
Infinity W201 Dual SIM 32 MB  preto/vermelho 32 MB RAM
Marca do celular: Samsung
Cor: Preto/Vermelho
Memória ram: ['32 MB RAM']
Espaço em hd: ['32 Mb']
1 ou 2 ship(s): ['2']
Preço: 100
Nome do Celular: Obasmart 3
Obabox ObaSmart 3 Dual SIM 16 GB  preto 1 GB RAM
Marca do celular: Samsung
Cor: Preto
Memória ram: ['1 GB RAM']
Espa

Nome do Celular: Poco X3 Nfc Dual Sim  Shadow
Xiaomi Poco X3 Nfc Dual Sim 64 Gb Shadow Gray 6 Gb Ram
Marca do celular: Samsung
Cor: Gray
Memória ram: ['6 Gb Ram']
Espaço em hd: ['64 Gb']
1 ou 2 ship(s): ['2']
Preço: 1.678
Nome do Celular: Lg K41S 4G Tela 6,55  3Gb Ram Android
Smartphone LG K41s 4g Tela 6,55 32gb 3gb Ram Android Titânio
Marca do celular: Lg
Cor: 3Gb
Memória ram: ['3gb Ram']
Espaço em hd: ['32Gb']
1 ou 2 ship(s): ['1']
Preço: 883
Nome do Celular: Lg K22 Tela 6,2  2Gb Ram Android
Smartphone LG K22 Tela 6,2 32gb 2gb Ram Android Titânio
Marca do celular: Lg
Cor: 2Gb
Memória ram: ['2gb Ram']
Espaço em hd: ['32Gb']
1 ou 2 ship(s): ['1']
Preço: 999
Nome do Celular: Samsung Galaxy M31 Tela 6.4  6Gb Ram
Smartphone Samsung Galaxy M31 Tela 6.4 128gb 6gb Ram Android
Marca do celular: Samsung
Cor: 128Gb
Memória ram: ['6gb Ram']
Espaço em hd: ['128Gb']
1 ou 2 ship(s): ['1']
Preço: 1.799
Nome do Celular: Lg K52 4G Tela 6,59  3Gb Ram
Smartphone LG K52 4g Tela 6,59 64 Gb 3gb Ram Vermelh

Nome do Celular: Samsung Galaxy A10S Tela De 6.2  2Gb
Smartphone Samsung Galaxy A10s Tela De 6.2 32gb 2gb Ram
Marca do celular: Samsung
Cor: 6.2
Memória ram: ['2gb Ram']
Espaço em hd: ['32Gb']
1 ou 2 ship(s): ['1']
Preço: 840
Nome do Celular: Redmi Note 8  4Gb Ram Versão Global +
Xiaomi Redmi Note 8 64gb 4gb Ram Versão Global + Nota
Marca do celular: Samsung
Cor: Versão
Memória ram: ['4gb Ram']
Espaço em hd: ['64Gb']
1 ou 2 ship(s): ['1']
Preço: 1.269
Nome do Celular: Multilaser E Lite Dual-Chip Android 8.1  Preto
Celular Multilaser E Lite Dual-chip Android 8.1 32gb Preto  
Marca do celular: Multilaser
Cor: Android
Memória ram: []
Espaço em hd: ['32Gb']
1 ou 2 ship(s): ['1']
Preço: 429
Nome do Celular: Xiaomi Redmi 9  + 4Gb Ram Versão
Smartphone Xiaomi Redmi 9 64gb + 4gb Ram Versão Global
Marca do celular: Xiaomi
Cor: 4Gb
Memória ram: ['4gb Ram']
Espaço em hd: ['64Gb']
1 ou 2 ship(s): ['1']
Preço: 1.088
Nome do Celular: Note 9S -  - Azul - 6Gb Ram - Versão
Xiaomi Note 9s - 128gb - Azul

Preço: 997
Nome do Celular: Positivo P36 Dual Sim 3G Tela 1,8
Celular Positivo P36 Dual Sim 3g Tela 1,8 128mb Ram 64mb
Marca do celular: Positivo
Cor: 1,8
Memória ram: ['128mb Ram']
Espaço em hd: ['64Mb']
1 ou 2 ship(s): ['2']
Preço: 299
Nome do Celular: Lg K52  Dual Chip Octa Core
Celular LG K52 64gb Dual Chip Octa Core Cinza
Marca do celular: Lg
Cor: Chip
Memória ram: []
Espaço em hd: ['64Gb']
1 ou 2 ship(s): ['1']
Preço: 1.199
Nome do Celular: Xiaomi Redmi 9 Prime  4Gb Novo, Lacrado,
Celular Xiaomi Redmi 9 Prime 128gb 4gb Novo, Lacrado, Nf-e
Marca do celular: Xiaomi
Cor: 4Gb
Memória ram: []
Espaço em hd: ['128Gb']
1 ou 2 ship(s): ['1']
Preço: 1.449
Nome do Celular: Semp Tcl L7  2Gb Ram Android 5,5
Smartphone Semp Tcl L7 32gb 2gb Ram Android 5,5 Polegadas
Marca do celular: Semp
Cor: Ram
Memória ram: ['2gb Ram']
Espaço em hd: ['32Gb']
1 ou 2 ship(s): ['1']
Preço: 1.018
Nome do Celular: Celular Para Idoso Dual Chip Facil Utilizar Up
Telefone Celular Para Idoso Dual Chip Facil Utilizar 

...

  '160'],
 ['Galaxy S10',
  'Samsung',
  'Preto-Prisma',
  ['8 GB RAM'],
  ['128 GB'],
  ['2'],
  '2.799'],
 ['Obasmart 3', 'Samsung', 'Ouro', ['1 GB RAM'], ['16 GB'], ['2'], '977'],
 ['Mi 10T 5G',
  'Samsung',
  'Prata-Lunar',
  ['6 GB RAM'],
  ['128 GB'],
  ['2'],
  '3.550'],
 ['Redmi 9 (Global)   Carbon',
  'Samsung',
  'Grey',
  ['3 GB RAM'],
  ['32 GB'],
  ['2'],
  '988'],
 ['Vita  32 Mb', 'Samsung', 'Preto', ['24 MB RAM'], ['32 MB'], ['2'], '106'],
 ['E7 Plus    Bronze',
  'Samsung',
  'Âmbar',
  ['4 GB RAM'],
  ['64 GB'],
  ['2'],
  '1.078'],
 ['Redmi 9 (Global)   Carbon',
  'Samsung',
  'Grey',
  ['3 GB RAM'],
  ['32 GB'],
  ['2'],
  '988'],
 ['Flip Up  32 Mb', 'Samsung', 'Preto', ['32 MB RAM'], ['32 MB'], ['2'], '155'],
 ['W201  32 Mb  Preto/',
  'Samsung',
  'Preto/Vermelho',
  ['32 MB RAM'],
  ['32 MB'],
  ['2'],
  '100'],
 ['Obasmart 3', 'Samsung', 'Preto', ['1 GB RAM'], ['16 GB'], ['2'], '997'],
 ['Galaxy S10',
  'Samsung',
  'Preto-Prisma',
  ['8 GB RAM'],
  ['128 GB'],
 

  '6,7',
  [],
  ['128gb'],
  ['1'],
  '2.099'],
 ['Samsung Galaxy A21S Dual Sim',
  'Samsung',
  'Azul',
  ['4 Gb Ram'],
  ['64 Gb'],
  ['2'],
  '1.449'],
 ['Poco X3 Nfc Dual Sim  Shadow',
  'Samsung',
  'Gray',
  ['6 Gb Ram'],
  ['64 Gb'],
  ['2'],
  '1.678'],
 ['Idosos Internet E Whatsapp Oba Smart 3 Gold',
  'Idosos',
  'Smart',
  [],
  [],
  ['1'],
  '997'],
 ['Flip  Idosos 2 Chips Radio Fm Camera',
  'Flip',
  'Radio',
  [],
  [],
  ['1'],
  '118'],
 ['Xiaomi Poco M3  6000 Mah + Capa +',
  'Xiaomi',
  '+',
  [],
  ['64gb'],
  ['1'],
  '1.216'],
 ['Motorola Moto G9 Power Dual  4Gb Ram',
  'Motorola',
  '128Gb',
  ['4gb Ram'],
  ['128gb'],
  ['1'],
  '1.699'],
 ['Galaxy Xcover Pro',
  'Motorola',
  'Preto',
  ['4 GB RAM'],
  ['64 GB'],
  ['2'],
  '1.490'],
 ['Smartphone Multilaser F P9130  Biometria 5,5',
  'Multilaser',
  'P9130',
  [],
  ['32gb'],
  ['1'],
  '529'],
 ['Samsung Galaxy A11 Dual 6.4   3Gb Ram',
  'Samsung',
  '64Gb',
  ['3gb Ram'],
  ['64gb'],
  ['1'],
  '1.298'],
 

 ['Samsung Galaxy A12 Dual 6.5  4Gb Ram',
  'Samsung',
  '64Gb',
  ['4gb Ram'],
  ['64gb'],
  ['1'],
  '1.349'],
 ['Galaxy A10S Dual Sim  Azul 2Gb Ram 6,2',
  'Samsung',
  '2Gb',
  ['2gb Ram'],
  ['32gb'],
  ['2'],
  '849'],
 ["Smartphone Multilaser G Max  4G 6.0'' Octa",
  'Multilaser',
  '4G',
  [],
  ['32gb'],
  ['1'],
  '669'],
 ['Xiaomi Redmi Note 9  Versão Global Lacrado',
  'Xiaomi',
  'Versão',
  [],
  ['128gb'],
  ['1'],
  '1.549'],
 ['Up Play Dual Chip Com', 'Up', 'Dual', [], [], ['1'], '84'],
 ['Lg K52 6.6   Lmk420Bmw 04 Câmeras Verde',
  'Lg',
  '04',
  [],
  ['64gb'],
  ['1'],
  '1.399'],
 ['Samsung Galaxy S10 Tela De 6.1  8Gb',
  'Samsung',
  '6.1',
  ['8gb Ram'],
  ['128gb'],
  ['1'],
  '2.799'],
 ['Obasmart3 Dourado Obabox + Fone + Película +',
  'Samsung',
  '+',
  [],
  ['32gb'],
  ['1'],
  '997'],
 ['Up Play Dual Sim 32 Mb',
  'Samsung',
  'Preto',
  ['32 Mb Ram'],
  ['32 Mb'],
  ['2'],
  '112'],
 ['Para Idoso Multilaser Up Play Dual  Preto',
  'Para',
  'Play',
  []

  ['1'],
  '2.488'],
 ['3G Teclado Grande Idoso 2 Chips Lanterna', '3G', '2', [], [], ['1'], '120'],
 ['Samsung Galaxy A02S Tela 6.5  3Gb Ram',
  'Samsung',
  '32Gb',
  ['3gb Ram'],
  ['32gb'],
  ['1'],
  '1.499'],
 ['Iphone 8 Plus Vitrine  Original Apple Loja Nfe',
  'Samsung',
  'Apple',
  [],
  ['64gb'],
  ['1'],
  '2.999'],
 ['Redmi 9I  / 4Gb Ram / Dual Sim / 4G / Tela 6.',
  'Samsung',
  '4G',
  ['4gb Ram'],
  ['128gb'],
  ['2'],
  '1.149'],
 ['2 Chips Teclado Grande Idoso Camera Mp3', '2', 'Idoso', [], [], ['1'], '70'],
 ['Samsung Galaxy J5 J500 Dual Chip',
  'Samsung',
  'Dual',
  [],
  ['16gb'],
  ['1'],
  '697'],
 ['Note 9S Aurora Blue 4Gb Ram  Rom.',
  'Samsung',
  '4Gb',
  ['4gb Ram'],
  ['64gb'],
  ['1'],
  '1.639'],
 ['Semp Tcl L5  1Gb Ram Android 5',
  'Semp',
  'Android',
  ['1gb Ram'],
  ['16gb', '16gb'],
  ['1'],
  '868'],
 ['Motorola Moto G8 Plus Xt2019  4Gb 6.3',
  'Motorola',
  '64Gb',
  [],
  ['64gb'],
  ['1'],
  '1.288'],
 ['Redmi Note 8 Dual Sim  Branco  +Nota',


  ['1'],
  '515'],
 ['X Vitrine  Original Appl Exposição 12X  S/',
  'Samsung',
  'Exposição',
  [],
  ['256gb'],
  ['1'],
  '3.840'],
 ['Xiaomi Redmi Poço X3  6Gb Versão',
  'Xiaomi',
  '64Gb',
  [],
  ['64gb'],
  ['1'],
  '1.999'],
 ['7 Plus  Gold(Vitrine) C/ Garantia E Nota',
  'Xiaomi',
  'Garantia',
  [],
  ['128gb'],
  ['1'],
  '2.700'],
 ['Xiaomi Poco M3 Dual Sim  Cool',
  'Xiaomi',
  'Blue',
  ['4 Gb Ram'],
  ['64 Gb'],
  ['2'],
  '1.276'],
 ['Galaxy A31  Câmera Quádrupla 48Mp+5Mp+ 8Mp +',
  'Galaxy',
  'Quádrupla',
  [],
  ['128gb'],
  ['1'],
  '1.549'],
 ['Samsung Galaxy A51 -', 'Samsung', 'A51', [], ['128 Gb'], ['1'], '1.999'],
 ['Redmi Note 8, ,4Gb Ram,Global+Capa Envio',
  'Samsung',
  '64Gb,4Gb',
  ['4gb Ram'],
  ['64gb'],
  ['1'],
  '1.318'],
 ['Samsung Galaxy A10S  Tela 6.2',
  'Samsung',
  '32Gb',
  [],
  ['32gb'],
  ['1'],
  '1.372'],
 ['Redmi 9 Dual Sim  Carbon',
  'Samsung',
  'Grey',
  ['4 Gb Ram'],
  ['64 Gb'],
  ['2'],
  '1.169'],
 ['Redmi Note 9S  6Gb Ram Versao

In [440]:
df_cel_mercado = pd.DataFrame(bloco_mercado_livre, columns=['Celular', 'Marca', 'cor', 'Ram', 'Hd','num_ship','Valor_real'])

In [441]:
df_cel_mercado.to_csv('dados_celphone_DF.csv')

In [442]:
df_cel_mercado.head()

,Celular,Marca,cor,Ram,Hd,num_ship,Valor_real
0,"Nokia C2 Nk011 Tl 5,7 (16+16 Cartão)1Gb",Nokia,32Gb,[1gb Ram],[32gb],[1],639
1,Galaxy A10S,Nokia,Preto,[2 GB RAM],[32 GB],[2],839
2,E6 Play,Nokia,Cinza-Metálico,[2 GB RAM],[32 GB],[2],810
3,Redmi Note 9S,Nokia,Azul,[4 GB RAM],[128 GB],[2],1.735
4,Redmi 9A,Nokia,Cinza,[2 GB RAM],[32 GB],[2],790


In [443]:
df_cel_mercado.Marca.value_counts()[:].head()

Samsung     225
Lg           77
Nokia        68
Motorola     59
Xiaomi       51
Name: Marca, dtype: int64

In [446]:
df_cel_mercado.Marca

0        Nokia
1        Nokia
2        Nokia
3        Nokia
4        Nokia
        ...   
604         Lg
605         Lg
606      Nokia
607    Samsung
608    Samsung
Name: Marca, Length: 609, dtype: object

In [327]:
for i in df_cel_mercado['Marca'] :
    if i == 'Celular':
        df_cel_mercado['Marca'].replace('x', re.findall(r'^\w+', df_cel_mercado['Celular'],re.M|re.I), inplace=True)
        print(df_cel_mercado['Marca'])
    else:
        pass
df_cel_mercado['Marca']

0      Smartphone
1         Samsung
2          Xiaomi
3          Xiaomi
4         Samsung
          ...    
604             x
605             x
606        Xiaomi
607    Smartphone
608          Moto
Name: Marca, Length: 609, dtype: object

In [328]:
df_cel_mercado.Marca.value_counts()

Smartphone    175
x             124
Xiaomi         88
Samsung        41
Iphone         39
Moto           25
Lg             18
Positivo       15
Multilaser     14
Motorola        7
Dl              6
Apple           5
Asus            5
Nokia           5
Cat             4
Blu             4
Smartprone      3
Realme          3
Huawei          3
Telefone        3
Quantum         2
Blackberry      2
Tcl             2
Alcatel         2
Smatphone       2
Obasmart        2
L8Star          1
Doogee          1
Aparelho        1
Google          1
Redmi           1
Galaxy          1
Ipro            1
Obabox          1
Lenovo          1
Infinity        1
Name: Marca, dtype: int64

In [267]:
d = list(df_cel_mercado.columns)

{i:df_cel_mercado[i].value_counts() for i in d} 

{'Celular': Lg K22 Tela 6,2  2Gb Ram Android                  12
 Lg K52 4G Tela 6,59  3Gb Ram                       8
 Lg K41S 4G Tela 6,55  3Gb Ram Android              7
 Samsung Galaxy M21S Tela 6.4  4Gb Ram -            6
 Lg K22 Plus Tela 6,2  3Gb Ram Quad-Core            6
                                                   ..
 Celular Para Idoso Dual Chip Facil Utilizar Up     1
 S60   Preto/                                       1
 Redmi Note 8 Pro /6Gb Versão Global+Nota           1
 K61                                                1
 Moto E6S  Dual Chip Tela 6.1                       1
 Name: Celular, Length: 361, dtype: int64,
 'Marca': Smartphone    175
 Celular       124
 Xiaomi         88
 Samsung        41
 Iphone         39
 Moto           25
 Lg             18
 Positivo       15
 Multilaser     14
 Motorola        7
 Dl              6
 Apple           5
 Asus            5
 Nokia           5
 Cat             4
 Blu             4
 Smartprone      3
 Realme          3
 

In [253]:
df_cel_mercado.shape

(609, 8)

In [233]:
df_cel_mercado.dtypes

Celular       object
Marca         object
cor           object
Ram           object
Hd            object
num_ship      object
Valor_real    object
x             object
dtype: object

In [226]:
df_cel_mercado.Marca.value_counts()

Smartphone    175
Celular       124
Xiaomi         88
Samsung        41
Iphone         39
Moto           25
Lg             18
Positivo       15
Multilaser     14
Motorola        7
Dl              6
Apple           5
Asus            5
Nokia           5
Cat             4
Blu             4
Smartprone      3
Realme          3
Huawei          3
Telefone        3
Quantum         2
Blackberry      2
Tcl             2
Alcatel         2
Smatphone       2
Obasmart        2
L8Star          1
Doogee          1
Aparelho        1
Google          1
Redmi           1
Galaxy          1
Ipro            1
Obabox          1
Lenovo          1
Infinity        1
Name: Marca, dtype: int64

In [225]:
df_cel_mercado.query('Hd=="1" & ')

Iphone        35
Celular       34
Smartphone    23
Xiaomi         8
Apple          5
Motorola       4
Samsung        3
Redmi          1
Blackberry     1
Name: Marca, dtype: int64

In [217]:
df_cel_mercado.Hd.value_counts()

1         114
32gb       66
64gb       62
128gb      61
32 GB      60
64 GB      38
128 GB     35
64 Gb      24
32 MB      19
32 Gb      13
16 GB      12
2          10
8 GB       10
997         9
169         6
16gb        4
128 Gb      3
151         3
165         2
118         2
98          2
229         2
429         2
1.199       2
70          2
256 GB      2
94          2
5.506       2
234         2
149         2
120         2
159         2
4.416       1
679         1
1.619       1
320         1
128 MB      1
512 Mb      1
116         1
249         1
1.336       1
84          1
2.698       1
24 MB       1
9.427       1
267         1
248         1
235         1
89          1
492         1
665         1
219         1
6 Gb        1
829         1
379         1
176         1
8 MB        1
129         1
1.999       1
Name: Hd, dtype: int64

In [85]:
len(df_num_nome)

359

In [88]:
df_num_nome = df_cel_mercado.Nome.value_counts()
df_num_nome = pd.DataFrame(df_num_nome, columns=['Nome'])
df_num_nome_maior_1 = df_num_nome.query('Nome >1')
df_num_nome_maior_1

,Nome
"LG K22 Tela 6,2 2gb Ram Android",8
"LG K52 4g Tela 6,59 3gb Ram",8
Samsung Galaxy A31 A315 Dual 4gb Ram,7
Samsung Galaxy A10s Tela De 6.2 2gb,7
Samsung Galaxy A12 Tela 6.5 4gb Ram,7
...,...
iPhone 8 Vitrine Dourado Garantia +,2
Galaxy A01 Core,2
Nokia 2.3 Nk005 Dual 4g 2gb Ram Hd+ 6.2,2
ZenFone Max Shot ZB634KL,2


In [38]:
lista_1 = ['R$ 9.900']
lista_2 = [',99']
str(lista_1).join(lista_2).replace(',','.')

'.99'

In [ ]:
str(lista_2).join(lista_1).replace('.',',')

In [ ]:
print(str(lista_2).join(lista_1)+str(lista_1).join(lista_2))

In [ ]:
bloco_zoom = []

for info in soup.findAll('div', {'class':'cardBody'}):
#     bloco = []
    
    info_nome = info.find('a').attrs['title']
    
    info_val = info.find('span', {'class':'mainValue'})
    
    info_cent = info.find('span', {'class':'centsValue'})
    
    info_nome_re = re.match(r'^\w+\s(\w+.)(\w+\s+\w+)', info_nome, re.M|re.I)

    info_val = info.find('span', {'class':'mainValue'})
  
    info_val_re = re.findall('(R[$]\s?[+-]?[0-9]{1,3}(?:(?:,?[0-9]{3}))*(?:\.[0-9]{1,2}\d+)?)', str(info_val))
    
    info_cent = info.find('span', {'class':'centsValue'})
    
    info_cent_re = re.findall('(.\s?[+-]?[0-9]{1,3}(?:(?:,?[0-9]{3}))*(?:\.[0-9]{1,2}\d+)?)', str(info_cent))
    bloco_zoom.append([info_nome,info_nome_re.group(1),info_nome_re.group(2),
                  str(info_cent_re).join(info_val_re)+str(info_val_re).join(info_cent_re)])
    
#     print(bloco)
    
#     print(info)
#     print(info_nome)
#     print(info_nome_re.group(1))
#     print(info_nome_re.group(2))
#     print(str(info_cent_re).join(info_val_re)+str(info_val_re).join(info_cent_re))

#     print('=============')
# print(valor)
# info_name = info.find('a').attrs['title']
# pprint(info_name)
print(len(bloco_zoom))
pprint(bloco_zoom)


In [ ]:
# Example of w+ and ^ Expression
import re
xx = "guru99,education is fun"
r1 = re.findall(r"^\w+",xx)
print (r1)

In [ ]:
# Example of \s expression in re.split function
import re
xx = "guru99,education is fun"
r1 = re.findall(r"^\w+", xx)
print (re.split(r'\s','we are splitting the words'))
print (re.split(r's','split the words'))

In [ ]:
# Example of re.M or Multiline Flags
import re
xx = """Guru99 gagre; 
careerguru99	
selenium"""
k1 = re.findall(r"^\w", xx)
k2 = re.findall(r"^\w", xx, re.MULTILINE)
k3 = re.findall("\\bg\w+\\b", xx)
k4 = re.findall("g\w+", xx)
k5 = re.findall("\\bg\w+\\b", xx, flags=re.I)

print (k1)
print (k2)
print (k3)
print (k4)
print (k5)

In [ ]:
date             = re.compile('(?:(?<!\:)(?<!\:\d)[0-3]?\d(?:st|nd|rd|th)?\s+(?:of\s+)?(?:jan\.?|january|feb\.?|february|mar\.?|march|apr\.?|april|may|jun\.?|june|jul\.?|july|aug\.?|august|sep\.?|september|oct\.?|october|nov\.?|november|dec\.?|december)|(?:jan\.?|january|feb\.?|february|mar\.?|march|apr\.?|april|may|jun\.?|june|jul\.?|july|aug\.?|august|sep\.?|september|oct\.?|october|nov\.?|november|dec\.?|december)\s+(?<!\:)(?<!\:\d)[0-3]?\d(?:st|nd|rd|th)?)(?:\,)?\s*(?:\d{4})?|[0-3]?\d[-\./][0-3]?\d[-\./]\d{2,4}', re.IGNORECASE)
time             = re.compile('\d{1,2}:\d{2} ?(?:[ap]\.?m\.?)?|\d[ap]\.?m\.?', re.IGNORECASE)
phone            = re.compile('''((?:(?<![\d-])(?:\+?\d{1,3}[-.\s*]?)?(?:\(?\d{3}\)?[-.\s*]?)?\d{3}[-.\s*]?\d{4}(?![\d-]))|(?:(?<![\d-])(?:(?:\(\+?\d{2}\))|(?:\+?\d{2}))\s*\d{2}\s*\d{3}\s*\d{4}(?![\d-])))''')
phones_with_exts = re.compile('((?:(?:\+?1\s*(?:[.-]\s*)?)?(?:\(\s*(?:[2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|(?:[2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?(?:[2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?(?:[0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(?:\d+)?))', re.IGNORECASE)
link             = re.compile('(?i)((?:https?://|www\d{0,3}[.])?[a-z0-9.\-]+[.](?:(?:international)|(?:construction)|(?:contractors)|(?:enterprises)|(?:photography)|(?:immobilien)|(?:management)|(?:technology)|(?:directory)|(?:education)|(?:equipment)|(?:institute)|(?:marketing)|(?:solutions)|(?:builders)|(?:clothing)|(?:computer)|(?:democrat)|(?:diamonds)|(?:graphics)|(?:holdings)|(?:lighting)|(?:plumbing)|(?:training)|(?:ventures)|(?:academy)|(?:careers)|(?:company)|(?:domains)|(?:florist)|(?:gallery)|(?:guitars)|(?:holiday)|(?:kitchen)|(?:recipes)|(?:shiksha)|(?:singles)|(?:support)|(?:systems)|(?:agency)|(?:berlin)|(?:camera)|(?:center)|(?:coffee)|(?:estate)|(?:kaufen)|(?:luxury)|(?:monash)|(?:museum)|(?:photos)|(?:repair)|(?:social)|(?:tattoo)|(?:travel)|(?:viajes)|(?:voyage)|(?:build)|(?:cheap)|(?:codes)|(?:dance)|(?:email)|(?:glass)|(?:house)|(?:ninja)|(?:photo)|(?:shoes)|(?:solar)|(?:today)|(?:aero)|(?:arpa)|(?:asia)|(?:bike)|(?:buzz)|(?:camp)|(?:club)|(?:coop)|(?:farm)|(?:gift)|(?:guru)|(?:info)|(?:jobs)|(?:kiwi)|(?:land)|(?:limo)|(?:link)|(?:menu)|(?:mobi)|(?:moda)|(?:name)|(?:pics)|(?:pink)|(?:post)|(?:rich)|(?:ruhr)|(?:sexy)|(?:tips)|(?:wang)|(?:wien)|(?:zone)|(?:biz)|(?:cab)|(?:cat)|(?:ceo)|(?:com)|(?:edu)|(?:gov)|(?:int)|(?:mil)|(?:net)|(?:onl)|(?:org)|(?:pro)|(?:red)|(?:tel)|(?:uno)|(?:xxx)|(?:ac)|(?:ad)|(?:ae)|(?:af)|(?:ag)|(?:ai)|(?:al)|(?:am)|(?:an)|(?:ao)|(?:aq)|(?:ar)|(?:as)|(?:at)|(?:au)|(?:aw)|(?:ax)|(?:az)|(?:ba)|(?:bb)|(?:bd)|(?:be)|(?:bf)|(?:bg)|(?:bh)|(?:bi)|(?:bj)|(?:bm)|(?:bn)|(?:bo)|(?:br)|(?:bs)|(?:bt)|(?:bv)|(?:bw)|(?:by)|(?:bz)|(?:ca)|(?:cc)|(?:cd)|(?:cf)|(?:cg)|(?:ch)|(?:ci)|(?:ck)|(?:cl)|(?:cm)|(?:cn)|(?:co)|(?:cr)|(?:cu)|(?:cv)|(?:cw)|(?:cx)|(?:cy)|(?:cz)|(?:de)|(?:dj)|(?:dk)|(?:dm)|(?:do)|(?:dz)|(?:ec)|(?:ee)|(?:eg)|(?:er)|(?:es)|(?:et)|(?:eu)|(?:fi)|(?:fj)|(?:fk)|(?:fm)|(?:fo)|(?:fr)|(?:ga)|(?:gb)|(?:gd)|(?:ge)|(?:gf)|(?:gg)|(?:gh)|(?:gi)|(?:gl)|(?:gm)|(?:gn)|(?:gp)|(?:gq)|(?:gr)|(?:gs)|(?:gt)|(?:gu)|(?:gw)|(?:gy)|(?:hk)|(?:hm)|(?:hn)|(?:hr)|(?:ht)|(?:hu)|(?:id)|(?:ie)|(?:il)|(?:im)|(?:in)|(?:io)|(?:iq)|(?:ir)|(?:is)|(?:it)|(?:je)|(?:jm)|(?:jo)|(?:jp)|(?:ke)|(?:kg)|(?:kh)|(?:ki)|(?:km)|(?:kn)|(?:kp)|(?:kr)|(?:kw)|(?:ky)|(?:kz)|(?:la)|(?:lb)|(?:lc)|(?:li)|(?:lk)|(?:lr)|(?:ls)|(?:lt)|(?:lu)|(?:lv)|(?:ly)|(?:ma)|(?:mc)|(?:md)|(?:me)|(?:mg)|(?:mh)|(?:mk)|(?:ml)|(?:mm)|(?:mn)|(?:mo)|(?:mp)|(?:mq)|(?:mr)|(?:ms)|(?:mt)|(?:mu)|(?:mv)|(?:mw)|(?:mx)|(?:my)|(?:mz)|(?:na)|(?:nc)|(?:ne)|(?:nf)|(?:ng)|(?:ni)|(?:nl)|(?:no)|(?:np)|(?:nr)|(?:nu)|(?:nz)|(?:om)|(?:pa)|(?:pe)|(?:pf)|(?:pg)|(?:ph)|(?:pk)|(?:pl)|(?:pm)|(?:pn)|(?:pr)|(?:ps)|(?:pt)|(?:pw)|(?:py)|(?:qa)|(?:re)|(?:ro)|(?:rs)|(?:ru)|(?:rw)|(?:sa)|(?:sb)|(?:sc)|(?:sd)|(?:se)|(?:sg)|(?:sh)|(?:si)|(?:sj)|(?:sk)|(?:sl)|(?:sm)|(?:sn)|(?:so)|(?:sr)|(?:st)|(?:su)|(?:sv)|(?:sx)|(?:sy)|(?:sz)|(?:tc)|(?:td)|(?:tf)|(?:tg)|(?:th)|(?:tj)|(?:tk)|(?:tl)|(?:tm)|(?:tn)|(?:to)|(?:tp)|(?:tr)|(?:tt)|(?:tv)|(?:tw)|(?:tz)|(?:ua)|(?:ug)|(?:uk)|(?:us)|(?:uy)|(?:uz)|(?:va)|(?:vc)|(?:ve)|(?:vg)|(?:vi)|(?:vn)|(?:vu)|(?:wf)|(?:ws)|(?:ye)|(?:yt)|(?:za)|(?:zm)|(?:zw))(?:/[^\s()<>]+[^\s`!()\[\]{};:\'".,<>?\xab\xbb\u201c\u201d\u2018\u2019])?)', re.IGNORECASE)
email            = re.compile("([a-z0-9!#$%&'*+\/=?^_`{|.}~-]+@(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?)", re.IGNORECASE)
ip               = re.compile('(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)', re.IGNORECASE)
ipv6             = re.compile('\s*(?!.*::.*::)(?:(?!:)|:(?=:))(?:[0-9a-f]{0,4}(?:(?<=::)|(?<!::):)){6}(?:[0-9a-f]{0,4}(?:(?<=::)|(?<!::):)[0-9a-f]{0,4}(?:(?<=::)|(?<!:)|(?<=:)(?<!::):)|(?:25[0-4]|2[0-4]\d|1\d\d|[1-9]?\d)(?:\.(?:25[0-4]|2[0-4]\d|1\d\d|[1-9]?\d)){3})\s*', re.VERBOSE|re.IGNORECASE|re.DOTALL)
price            = re.compile('[$]\s?[+-]?[0-9]{1,3}(?:(?:,?[0-9]{3}))*(?:\.[0-9]{1,2})?')
hex_color        = re.compile('(#(?:[0-9a-fA-F]{8})|#(?:[0-9a-fA-F]{3}){1,2})\\b')
credit_card      = re.compile('((?:(?:\\d{4}[- ]?){3}\\d{4}|\\d{15,16}))(?![\\d])')
btc_address      = re.compile('(?<![a-km-zA-HJ-NP-Z0-9])[13][a-km-zA-HJ-NP-Z0-9]{26,33}(?![a-km-zA-HJ-NP-Z0-9])')
street_address   = re.compile('\d{1,4} [\w\s]{1,20}(?:street|st|avenue|ave|road|rd|highway|hwy|square|sq|trail|trl|drive|dr|court|ct|park|parkway|pkwy|circle|cir|boulevard|blvd)\W?(?=\s|$)', re.IGNORECASE)
zip_code         = re.compile(r'\b\d{5}(?:[-\s]\d{4})?\b')
po_box           = re.compile(r'P\.? ?O\.? Box \d+', re.IGNORECASE)
ssn              = re.compile('(?!000|666|333)0*(?:[0-6][0-9][0-9]|[0-7][0-6][0-9]|[0-7][0-7][0-2])[- ](?!00)[0-9]{2}[- ](?!0000)[0-9]{4}')

In [ ]:
import regex

In [ ]:
regex.match(r'(?(?=\d)\d+|\w+)', '123abc')

In [ ]:
regex.findall(r'(?(?=\d)\d+|\w+)', '123abc36 7gdh127')

In [ ]:
regex.match(r'(?(?=\d)\d+|\w+)', 'abc123')

In [ ]:
regex.findall(r'(?(?=\d)\d+|\w+)', '123abc')

In [ ]:
# Using re.findall for text
import re

list = ["guru99 get", "guru99 give", "guru Selenium"]
for element in list:
    z = re.match("(g\w+)\W(g\w+)", element)
if z:
    print(z.groups())
    
patterns = ['software testing', 'guru99']
text = 'software testing is fun?'
for pattern in patterns:
    print 'Looking for "%s" in "%s" ->' % (pattern, text),
    if re.search(pattern, text):
        print ('found a match!')
else:
    print ('no match')
abc = 'guru99@google.com, careerguru99@hotmail.com, users@yahoomail.com'
emails = re.findall(r'[\w\.-]+@[\w\.-]+', abc)
for email in emails:
    print (email)


<strong><b><font size="5">Data Science Academy</font></b></strong>

<strong><b><font size="5">Web Scraping e Análise de Dados</font></b></strong>

<strong><b><font size="5">Lab 2</font></b></strong>

<strong><b><font size="4">Web Scraping e Análise Estatística - Avaliando o Desempenho dos Participantes em Corridas de 10 KM com Dados de Sensores RFID</font></b></strong>

![title](imagens/lab2.png)

## Definição do Problema

Você gosta de correr? Corrida é um dos esportes mais praticados em todo mundo, além de uma atividade saudável e prazerosa.

Neste Lab vamos coletar dados reais de participantes de uma corrida de 10 KM realizada em Hillsboro no Oregon, EUA, em junho de 2017. 

Resumo da corrida:

- Pessoas que completaram a corrida: 577
- Homens: 414
- Mulheres: 163

O objetivo desta análise é encontrar:
    
- O tempo médio de conclusão da corrida.
- Se os tempos de chegada dos corredores seguem uma distribuição normal.
- Analisar se há diferença de desempenho entre homens e mulheres de várias faixas etárias.

Os tempos registrados na corrida (bem como outros dados) são bem precisos pois os participantes usaram sensores RFID de onde os dados foram extraídos e disponibilizados na web. Mais detalhes sobre o uso dos sensores RFID aqui:

https://www.hubertiming.com/runners/faq

## Fonte de Dados

Como fonte de dados usaremos o web site <a href="https://www.hubertiming.com/">Huber Timing</a> que contém o resultado das corridas que acontecem nos EUA, bem como a agenda de novos eventos.

Aqui a página com o resultado da corrida que faremos a análise:

https://www.hubertiming.com/results/2017GPTR10K

## Carregando os Pacotes Usados Neste Jupyter Notebook

In [ ]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

In [ ]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook, se necessário.

In [ ]:
# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [ ]:
# Usaremos o pacote Pingouin para algumas das análises estatísticas
# https://pingouin-stats.org/index.html
!pip install -q pingouin

In [ ]:
# Imports
import re
import requests
import matplotlib as m
import numpy as np
import pandas as pd
import pingouin as pg
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
from scipy import stats

# Imports para formatação dos gráficos
sns.set_style('whitegrid')
%matplotlib inline

In [ ]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

## Web Scraping

In [ ]:
# Definindo a URL com o resultado da corrida
url = 'https://www.hubertiming.com/results/2017GPTR10K'

In [ ]:
# Abre a conexão com a url
html = urlopen(url)

In [ ]:
# Extrai o código HTML
soup = bs(html,'lxml')

In [ ]:
# Extrai o texto do código html
texto = soup.get_text()

In [ ]:
# Vamos visualizar as 10 primeiras linhas
linhas = soup.find_all('tr')
print(linhas[:10])

## Limpeza e Transformação

Agora começa o trabalho pesado. Não fique apenas executando código e compreenda o que está sendo feito. Se tiver dificuldades, pergunte! Temos vários canais de suporte a sua disposição.

Começaremos obtendo todas as linhas da tabela em forma de lista e depois converteremos essa lista em um dataframe para facilitar a manipulação.

In [ ]:
# Loop
for linha in linhas:
    linha_td = linha.find_all('td')

In [ ]:
print(linha_td)

Com expressões regulares em Python, vamos extrair os dados das tags HTML.

In [ ]:
# Lista vazia para receber as linhas
lista_linhas = []

In [ ]:
# Loop
for linha in linhas:
    cells = linha.find_all('td')
    str_cells = str(cells)
    clean = re.compile('<.*?>')
    clean2 = (re.sub(clean, '', str_cells))
    lista_linhas.append(clean2)

In [ ]:
print(clean2)

Convertemos a lista em um dataframe e obtemos uma visão rápida das 10 primeiras linhas usando o Pandas.

In [ ]:
# Convertemos a lista em dataframe
df = pd.DataFrame(lista_linhas)

In [ ]:
df.head(10)

Claramente precisamos limpar os dados. Vamos ao trabalho.

In [ ]:
# Dividindo cada linha e separando os dados a cada vírgula
df1 = df[0].str.split(',', expand = True)

In [ ]:
df1.head(10)

Está começando a ficar bonito. Vamos fazer mais algumas limpezas. Leia e compreenda o código. Se tiver dúvidas, pergunte (embora isso não deva ser novidade se você fez o curso gratuito de <a href="https://www.datascienceacademy.com.br/course?courseid=python-fundamentos">Python Fundamentos</a> aqui na DSA).

In [ ]:
# Removemos colchetes de abertura (lado esquerdo) do texto
df1[0] = df1[0].str.strip('[')

In [ ]:
df1.head(10)

In [ ]:
# Extraímos o cabeçalho de todas as tabelas no código HTML
col_labels = soup.find_all('th')

In [ ]:
col_labels

In [ ]:
# Extraímos texto entre tags html dos cabeçalhos de tabela (extraímos o nome)
all_header = []
col_str = str(col_labels)
cleantext2 = bs(col_str, "lxml").get_text()
all_header.append(cleantext2)

In [ ]:
print(all_header)

In [ ]:
# Convertendo a lista de cabeçalhos em um dataframe do pandas
df2 = pd.DataFrame(all_header)

In [ ]:
df2.head()

In [ ]:
# Dividindo a coluna "0" em várias colunas na posição de vírgula para todas as linhas
df3 = df2[0].str.split(',', expand = True)

In [ ]:
df3.head()

In [ ]:
# Concatenando os 2 dataframes
frames = [df3, df1]
df4 = pd.concat(frames)

In [ ]:
df4.head(10)

In [ ]:
# Atribuindo a primeira coluna como cabeçalho
df5 = df4.rename(columns = df4.iloc[0])

In [ ]:
df5.head()

In [ ]:
# Shape
df5.shape

In [ ]:
# Obtendo uma visão geral dos dados para análise
df5.info()

In [ ]:
# Verificando valores ausentes
df5.isnull().sum()

In [ ]:
# Removendo todas as linhas com valores ausentes
df6 = df5.dropna(axis = 0, how = 'any')

In [ ]:
# Descartando o cabeçalho da tabela replicada como a primeira linha no df5
df7 = df6.drop(df6.index[0])

In [ ]:
# Renomeando as colunas [Place e Team] 
df7.rename(columns = {'[Place': 'Place'}, inplace = True)
df7.rename(columns = {' Team]': 'Team'}, inplace = True)

In [ ]:
# Limpeza final dos dados - remoção do colchete de fechamento das células na coluna "Team".
df7['Team'] = df7['Team'].str.strip(']')

In [ ]:
df7.head()

## Análise de Dados

In [ ]:
# Convertendo a coluna "Chip Time" em apenas minutos
# Chip time é o tempo total de corrida medido com a leitura do sensor RFID no selo da camisa do participante
time_list = df7[' Chip Time'].tolist()

In [ ]:
# Visualizando uma amostra de dados
time_list[1:5]

In [ ]:
# Lista para receber o resultado da conversão
time_mins = []

In [ ]:
# Loop para conversão
for i in time_list:
    i = i.strip(' ')
    if len(i)!=7:
        i = '0:' + i
    h, m, s = i.split(':')
    math = (int(h) * 3600 + int(m) * 60 + int(s))/60
    time_mins.append(math)

Convertendo a lista novamente em um dataframe e criamos uma nova coluna ("Runner_mins") como tempo total de corrida, mas agora em minutos.

In [ ]:
# Nova coluna
df7['Runner_mins'] = time_mins

In [ ]:
df7.head()

In [ ]:
df7.dtypes

In [ ]:
# Cálculo de estatísticas para colunas numéricas no dataframe
df7.describe(include = [np.number])

**Análise**:

- O tempo médio de chip para todos os corredores foi de ~ 60 minutos (a média na tabela acima).
- O corredor 10K mais rápido terminou em 36,35 minutos (valor mínimo na tabela acima).
- O corredor mais lento terminou em 101,30 minutos (valor máximo na tabela acima).

Vamos criar um boxplot para sumarizar nossa análise.

In [ ]:
# Plot
ax = sns.boxplot(x = df7["Runner_mins"], palette = "Set3", orient = "v")
plt.ylabel('Chip Time')
plt.xlabel('Runners')
plt.show()

Acima estão as estatísticas de resumo de dados para os corredores mostrados em um boxplot: Máximo, mínimo, médio, primeiro quartil, terceiro quartil, incluindo valores extremos.

Criaremos agora um gráfico de distribuição dos tempos dos chips dos corredores usando a biblioteca seaborn.

In [ ]:
# Plot
x = df7['Runner_mins']
ax = sns.distplot(x, hist = True, kde = True, rug = False, color = 'blue', bins = 25, hist_kws = {'edgecolor':'black'})
plt.show()

A distribuição da variável parece quase normal. Mas vamos realizar um teste estatístico para confirmar.

Um ponto de decisão importante ao trabalhar com uma amostra de dados é usar métodos estatísticos paramétricos ou não paramétricos.

Os métodos estatísticos paramétricos assumem que os dados têm uma distribuição conhecida e específica, geralmente uma distribuição gaussiana (normal). Se uma amostra de dados não é gaussiana, as suposições dos testes estatísticos paramétricos são violadas e métodos estatísticos não paramétricos devem ser utilizados.

Há várias técnicas que você pode usar para verificar se sua amostra de dados se desvia de uma distribuição gaussiana, chamada testes de normalidade.

O pacote stats do SciPy fornece uma função para um dos testes de normalidade mais comuns, o Shapiro-Wilk Test. Usaremos ainda a função normality do pacote Pengouin.

O Shapiro-Wilk é um teste de hipóteses, que testa essas duas hipóteses:

**A hipótese nula deste teste é que a população é normalmente distribuída. Assim, se o valor-p for menor que o nível alfa escolhido (normalmente definido em 0,05), a hipótese nula será rejeitada e há evidências de que os dados testados não são normalmente distribuídos.**

O resultado do teste de Shapiro-Wilk deve ser interpretado com cautela no caso de amostras grandes. 

In [ ]:
# Valor de x
x = df7['Runner_mins']

In [ ]:
# Teste de normalidade com Scipy
stats.shapiro(x)

In [ ]:
# Teste de normalidade com Pingouin
pg.normality(x)

**Análise**:

- A variável não segue uma distribuição normal, uma vez que o valor-p é menor que 0.05 e assim há evidências estatísticas para rejeitar a hipótese nula. A função normality() do pacote Pingouin coloca essa informação na última coluna.

Vamos agora descobrir se houve alguma diferença de desempenho entre homens e mulheres de várias faixas etárias.

In [ ]:
# Separando os dados
f_runners = df7.loc[df7[' Gender'] == ' F']['Runner_mins']
m_runners = df7.loc[df7[' Gender'] == ' M']['Runner_mins']

In [ ]:
# Criaremos os 2 plots em uma só figura para facilitar a comparação
sns.distplot(f_runners, hist = True, kde = True, rug = False, hist_kws = {'edgecolor':'black'}, label = 'Mulher')
sns.distplot(m_runners, hist = False, kde = True, rug = False, hist_kws = {'edgecolor':'black'}, label = 'Homem')
plt.legend()

**Análise**:

- A distribuição indica que as mulheres eram mais lentas que os homens, em média.

In [ ]:
# Computando estatísticas resumidas para homens e mulheres separadamente usando o método groupby ()
g_stats = df7.groupby(" Gender", as_index = True).describe()
g_stats

**Análise**:
    
- O tempo médio de chip para todas as mulheres e homens foi de ~ 66 minutos e ~ 58 minutos, respectivamente, comprovando a análise do gráfico. 

Um boxplot ajuda a comparar as amostras de homens e mulheres.

In [ ]:
# Plot
df7.boxplot(column = 'Runner_mins', by = ' Gender')
plt.ylabel('Chip Time')
plt.suptitle("")

Nossa fonte de dados possui resultados de diversas corridas. Material em abundância para você praticar seus conhecimentos em análise de dados.

# Fim